In [1]:
# import libraries
import datetime
from datetime import date, timedelta
from os import path
import os
import pandas as pd
import numpy as np 
import warnings
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import sys
import boto3
import s3fs
from fastparquet import ParquetFile, write
sys.path.append("../HistoricalData/")
from getData import get_data
# from getDataLocal import get_data

from sklearn.preprocessing import Imputer
imputer = Imputer()

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)

In [2]:
datafolder = "/Users/apaul2/Documents/_Common/capstone/Project/data"

In [3]:
def timeOfDay(hour):
    if int(hour) >=0 and int(hour) < 6:
        return 'night'
    elif int(hour) >=6 and int(hour) < 12:
        return 'morning'
    elif int(hour) >=12 and int(hour) < 18:
        return 'afternoon'
    elif int(hour) >=18 and int(hour) < 24:
        return 'evening'
    else:
        return 'other'

In [4]:
# define direction degree range by rounding to nearest cardinal direction
# NORTH = (316,45)
# EAST = (46,135)
# SOUTH = (136,225)
# WEST = (226,315)

# tried a reduced range of only +/- 15 degrees instead of 45 degrees
NORTH = (346,15)
EAST = (76,105)
SOUTH = (166,195)
WEST = (256,285)

In [5]:
sep_df = pd.read_parquet("{}/monthly/201910.parquet".format(datafolder))

In [13]:
sep_df.head()

,0_3um,0_5um,10_0um,1_0um,2_5um,5_0um,a_h,agency_name,aqi,call_sign,call_sign2,call_sign3,category,city,county,created,created_at,day,device_loc_typ,epa_pm25_unit,epa_pm25_value,full_aqs_code,gust_speed,gusts,hidden,high_reading_flag,hour,humidity,interval,intl_aqs_code,is_owner,lat,lon,minute,month,parent_id,pm10_0,pm10_0_atm,pm1_0,pm1_0_atm,pm2_5_atm,pm2_5_cf_1,raw_concentration,report_modifier,rssi,sensor_id,sensor_name,site_name,sys_maint_reqd,temperature,thingspeak_primary_id,thingspeak_primary_id_read_key,thingspeak_secondary_id,thingspeak_secondary_id_read_key,uptime,variable_wind_info,variable_winds,wban_number,wind_data,wind_direction,wind_speed,year,zipcode,zulu_time
0,131.87,39.33,0.00,7.93,1.43,0.63,None,San Francisco Bay Area AQMD,15.000000,KSFO,SFO,KSFO,1.0,San Francisco,San Francisco County,201910010100,2019/10/01T01:00,01,outside,UG/M3,3.600000,060750005,NaN,0.0,false,nan,01,53.0,5-MIN,840060750005,0,37.72244,-122.439302,00,10,nan,0.80,0.80,0.00,0.00,0.49,0.49,4.0,,-67.0,16939,#SAFQ11,San Francisco,0.0,66.0,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,3974.0,,0.0,23234,1.0,270,9.0,2019,94112,010900Z
1,218.38,60.75,0.00,10.07,1.76,0.00,None,San Francisco Bay Area AQMD,15.166667,KSFO,SFO,KSFO,1.0,San Francisco,San Francisco County,201910010110,2019/10/01T01:10,01,outside,UG/M3,3.633333,060750005,NaN,0.0,false,nan,01,55.0,5-MIN,840060750005,0,37.72244,-122.439302,10,10,nan,0.78,0.78,0.22,0.22,0.78,0.78,4.0,,-70.0,16939,#SAFQ11,San Francisco,0.0,66.0,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,3984.0,,0.0,23234,1.0,270,9.0,2019,94112,010910Z
2,191.91,54.15,0.85,14.63,3.88,1.74,None,San Francisco Bay Area AQMD,15.333333,KSFO,SFO,KSFO,1.0,San Francisco,San Francisco County,201910010120,2019/10/01T01:20,01,outside,UG/M3,3.666667,060750005,NaN,0.0,false,nan,01,56.0,5-MIN,840060750005,0,37.72244,-122.439302,20,10,nan,2.13,2.13,0.18,0.18,1.06,1.06,4.0,,-68.0,16939,#SAFQ11,San Francisco,0.0,65.0,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,3994.0,,0.0,23234,1.0,270,8.0,2019,94112,010920Z
3,234.97,66.09,0.41,16.88,4.85,0.76,None,San Francisco Bay Area AQMD,15.500000,KSFO,SFO,KSFO,1.0,San Francisco,San Francisco County,201910010130,2019/10/01T01:30,01,outside,UG/M3,3.700000,060750005,NaN,0.0,false,nan,01,57.0,5-MIN,840060750005,0,37.72244,-122.439302,30,10,nan,2.06,2.06,0.10,0.10,1.51,1.51,4.0,,-73.0,16939,#SAFQ11,San Francisco,0.0,65.0,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,4004.0,,0.0,23234,1.0,270,8.0,2019,94112,010930Z
4,219.72,63.83,0.00,13.31,3.86,1.27,None,San Francisco Bay Area AQMD,15.666667,KSFO,SFO,KSFO,1.0,San Francisco,San Francisco County,201910010140,2019/10/01T01:40,01,outside,UG/M3,3.733333,060750005,NaN,0.0,false,nan,01,58.0,5-MIN,840060750005,0,37.72244,-122.439302,40,10,nan,2.15,2.15,0.35,0.35,1.23,1.23,4.0,,-68.0,16939,#SAFQ11,San Francisco,0.0,65.0,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,4014.0,,0.0,23234,1.0,270,7.0,2019,94112,010940Z


In [7]:
sep_df.year.unique(), sep_df.month.unique()

(array(['2019'], dtype=object), array(['10'], dtype=object))

In [3]:
full_df = pd.read_parquet("{}/201809_201909.parquet".format(datafolder))

In [5]:
full_df.describe().T

,count,mean,std,min,25%,50%,75%,max
0_3um,7543001.0,1.671287e+03,1.563826e+06,0.000000e+00,3.123900e+02,6.008500e+02,9.824800e+02,4.294967e+09
0_5um,7543001.0,8.739901e+02,1.563824e+06,0.000000e+00,9.040000e+01,1.751700e+02,2.866900e+02,4.294967e+09
10_0um,7543001.0,7.818764e+02,1.649512e+06,0.000000e+00,1.700000e-01,3.600000e-01,7.200000e-01,4.294967e+09
1_0um,7543001.0,6.240709e+02,1.563824e+06,0.000000e+00,1.370000e+01,2.939000e+01,5.340000e+01,4.294967e+09
2_5um,7543001.0,7.660134e+02,1.648415e+06,0.000000e+00,1.280000e+00,3.080000e+00,6.450000e+00,4.294967e+09
5_0um,7543001.0,7.832197e+02,1.649512e+06,0.000000e+00,3.800000e-01,9.000000e-01,2.000000e+00,4.294967e+09
aqi,7529877.0,2.464180e+01,8.673212e+01,-9.990000e+02,1.833333e+01,2.900000e+01,4.233333e+01,2.885000e+02
category,7529877.0,1.178321e+00,4.813874e-01,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e+00
created,7630368.0,2.018936e+11,3.153542e+07,2.018090e+11,2.019022e+11,2.019052e+11,2.019073e+11,2.019093e+11
epa_pm25_value,7529877.0,8.623098e+00,1.092051e+01,-2.900000e+00,4.400000e+00,6.900000e+00,1.013333e+01,2.387833e+02


In [8]:
full_df.columns

Index(['0_3um', '0_5um', '10_0um', '1_0um', '2_5um', '5_0um', 'a_h',
       'agency_name', 'aqi', 'call_sign', 'call_sign2', 'call_sign3',
       'category', 'city', 'county', 'created', 'created_at', 'day',
       'device_loc_typ', 'epa_pm25_unit', 'epa_pm25_value', 'full_aqs_code',
       'gust_speed', 'gusts', 'hidden', 'high_reading_flag', 'hour',
       'humidity', 'interval', 'intl_aqs_code', 'is_owner', 'lat', 'lon',
       'minute', 'month', 'parent_id', 'pm10_0', 'pm10_0_atm', 'pm1_0',
       'pm1_0_atm', 'pm2_5_atm', 'pm2_5_cf_1', 'raw_concentration',
       'report_modifier', 'rssi', 'sensor_id', 'sensor_name', 'site_name',
       'sys_maint_reqd', 'temperature', 'thingspeak_primary_id',
       'thingspeak_primary_id_read_key', 'thingspeak_secondary_id',
       'thingspeak_secondary_id_read_key', 'uptime', 'variable_wind_info',
       'variable_winds', 'wban_number', 'wind_data', 'wind_direction',
       'wind_speed', 'year', 'zipcode', 'zulu_time'],
      dtype='object')

In [9]:
full_df[['sensor_id', 'parent_id']].describe()

,sensor_id,parent_id
count,7630368,7630368
unique,546,235
top,4481,nan
freq,23341,4280699


In [8]:
aug_df = full_df[(full_df.year=='2019') & (full_df.month=='08')]

In [9]:
len(aug_df), aug_df.year.unique(), aug_df.month.unique()

(904116, array(['2019'], dtype=object), array(['08'], dtype=object))

In [10]:
jan_df = full_df[(full_df.year=='2019') & (full_df.month=='01')]
feb_df = full_df[(full_df.year=='2019') & (full_df.month=='02')]
mar_df = full_df[(full_df.year=='2019') & (full_df.month=='03')]
apr_df = full_df[(full_df.year=='2019') & (full_df.month=='04')]
may_df = full_df[(full_df.year=='2019') & (full_df.month=='05')]
jun_df = full_df[(full_df.year=='2019') & (full_df.month=='06')]
jul_df = full_df[(full_df.year=='2019') & (full_df.month=='07')]

In [11]:
sep18_df = full_df[(full_df.year=='2018') & (full_df.month=='09')]
oct18_df = full_df[(full_df.year=='2018') & (full_df.month=='10')]
nov18_df = full_df[(full_df.year=='2018') & (full_df.month=='11')]
dec18_df = full_df[(full_df.year=='2018') & (full_df.month=='12')]

In [8]:
df = deepcopy(sep_df)
df.reset_index(inplace=True, drop=True)

In [9]:
# remove rows with na data for 2_5um
df = df[df['2_5um'].notna()]
len(df)

740689

In [10]:
df = df[df.sys_maint_reqd != 1]
len(df)

714428

In [11]:
df = df[df.high_reading_flag != 1]
len(df)

714428

In [12]:
df = df[df.device_loc_typ == 'outside']
len(df)

714428

In [14]:
df['wkday'] = df['created_at'].apply(lambda x: datetime.datetime.strptime(x,"%Y/%m/%dT%H:%M").strftime("%w"))
df['daytype'] = df['wkday'].apply(lambda x: 'Weekend' if x in ('0','6') else 'Weekday')
df['timeofday'] = df['hour'].apply(lambda x: timeOfDay(x))

In [229]:
# df.wind_direction.unique()

In [15]:
# go through the dataframe and add new categorical column that indicates direction: 
# North, South, East, West, No wind, Missing, ERROR

wind_compass = [] 
for row in range(len(df)):
    try:
        degree = int(df.loc[row].wind_direction)
    except Exception as e:
        wind_compass.append('Missing')
        continue
    if df.loc[row].wind_speed == 0:
        wind_compass.append('No wind')
    elif degree >= NORTH[0] or degree <= NORTH[1]:
        wind_compass.append('North')
    elif degree >= EAST[0] and degree <= EAST[1]:
        wind_compass.append('East')
    elif degree >= SOUTH[0] and degree <= SOUTH[1]:
        wind_compass.append('South')
    elif degree >= WEST[0] and degree <= WEST[1]:
        wind_compass.append('West')
    else:
        wind_compass.append('ERROR')
df['wind_compass'] = wind_compass

In [16]:
df.wind_compass.unique()

array(['West', 'ERROR', 'No wind', 'North', 'East', 'South', 'Missing'],
      dtype=object)

In [17]:
parquet_file = "{}/monthly/201910.parquet".format(datafolder)
write(parquet_file, df,compression='GZIP')

In [83]:
# tstdf = pd.read_parquet("{}/monthly/201909.parquet".format(datafolder))

In [233]:
df.head()

,0_3um,0_5um,10_0um,1_0um,2_5um,5_0um,a_h,agency_name,aqi,call_sign,call_sign2,call_sign3,category,city,county,created,created_at,day,device_loc_typ,epa_pm25_unit,epa_pm25_value,full_aqs_code,gust_speed,gusts,hidden,high_reading_flag,hour,humidity,interval,intl_aqs_code,is_owner,lat,lon,minute,month,parent_id,pm10_0,pm10_0_atm,pm1_0,pm1_0_atm,pm2_5_atm,pm2_5_cf_1,raw_concentration,report_modifier,rssi,sensor_id,sensor_name,site_name,sys_maint_reqd,temperature,thingspeak_primary_id,thingspeak_primary_id_read_key,thingspeak_secondary_id,thingspeak_secondary_id_read_key,uptime,variable_wind_info,variable_winds,wban_number,wind_data,wind_direction,wind_speed,year,zipcode,zulu_time,wkday,daytype,timeofday,wind_compass
180,368.92,107.15,0.26,23.18,4.44,1.28,None,San Francisco Bay Area AQMD,24.000000,KOAK,OAK,KOAK,1.0,San Francisco,San Francisco County,201812010000,2018/12/01T00:00,01,outside,UG/M3,5.800000,60750005.0,NaN,0.0,None,nan,00,48.0,5-MIN,8.400608e+11,0,37.78119,-122.476532,00,12,nan,3.08,3.08,0.82,0.82,2.31,2.31,7.000000,,-72.0,4101,17th Ave,San Francisco,0.0,65.0,358843,REEYEYHCF3H76UDM,358844,YA16CUCUP7RVYE9A,71.0,,0.0,23230,1.0,000,0.0,2018,94121,010800Z,6,Weekend,night,Missing
181,333.86,95.90,0.24,16.93,3.45,1.29,None,San Francisco Bay Area AQMD,24.833333,KOAK,OAK,KOAK,1.0,San Francisco,San Francisco County,201812010010,2018/12/01T00:10,01,outside,UG/M3,5.983333,60750005.0,NaN,0.0,None,nan,00,49.0,5-MIN,8.400608e+11,0,37.78119,-122.476532,10,12,nan,2.64,2.64,0.86,0.86,1.79,1.79,7.166667,,-69.0,4101,17th Ave,San Francisco,0.0,65.0,358843,REEYEYHCF3H76UDM,358844,YA16CUCUP7RVYE9A,80.0,,0.0,23230,1.0,090,3.0,2018,94121,010810Z,6,Weekend,night,Missing
182,365.57,106.67,0.16,18.36,2.07,0.50,None,San Francisco Bay Area AQMD,25.666667,KOAK,OAK,KOAK,1.0,San Francisco,San Francisco County,201812010020,2018/12/01T00:20,01,outside,UG/M3,6.166667,60750005.0,NaN,0.0,None,nan,00,48.0,5-MIN,8.400608e+11,0,37.78119,-122.476532,20,12,nan,2.38,2.38,0.95,0.95,1.98,1.98,7.333333,,-64.0,4101,17th Ave,San Francisco,0.0,66.0,358843,REEYEYHCF3H76UDM,358844,YA16CUCUP7RVYE9A,91.0,,0.0,23230,1.0,090,3.0,2018,94121,010820Z,6,Weekend,night,Missing
183,347.77,103.57,0.55,22.85,4.22,2.17,None,San Francisco Bay Area AQMD,26.500000,KOAK,OAK,KOAK,1.0,San Francisco,San Francisco County,201812010030,2018/12/01T00:30,01,outside,UG/M3,6.350000,60750005.0,NaN,0.0,None,nan,00,49.0,5-MIN,8.400608e+11,0,37.78119,-122.476532,30,12,nan,3.58,3.58,1.05,1.05,2.28,2.28,7.500000,,-65.0,4101,17th Ave,San Francisco,0.0,66.0,358843,REEYEYHCF3H76UDM,358844,YA16CUCUP7RVYE9A,100.0,,0.0,23230,1.0,090,3.0,2018,94121,010830Z,6,Weekend,night,Missing
184,320.29,95.62,0.24,21.26,3.60,1.07,None,San Francisco Bay Area AQMD,27.333333,KOAK,OAK,KOAK,1.0,San Francisco,San Francisco County,201812010040,2018/12/01T00:40,01,outside,UG/M3,6.533333,60750005.0,NaN,0.0,None,nan,00,49.0,5-MIN,8.400608e+11,0,37.78119,-122.476532,40,12,nan,2.98,2.98,0.69,0.69,1.88,1.88,7.666667,,-61.0,4101,17th Ave,San Francisco,0.0,65.0,358843,REEYEYHCF3H76UDM,358844,YA16CUCUP7RVYE9A,111.0,,0.0,23230,1.0,080,4.0,2018,94121,010840Z,6,Weekend,night,Missing
